# AI-Powered SQL Query Chatbot for Odoo PostgreSQL Database

## Problem Statement

- **Objective**: 
  Build a conversational AI-powered chatbot that generates and executes SQL queries on a PostgreSQL database for an Odoo-based system. The chatbot should interpret natural language queries and return relevant data from multiple tables.

- **Key Requirements**:
  - Parse user queries in natural language.
  - Identify relevant tables and columns.
  - Dynamically generate SQL queries to retrieve data.
  - Handle SQL operations such as joins, filtering, and aggregations.
  - Provide conversational responses with correct results.
  - Efficiently handle large datasets and ambiguous queries.

## Workflow Overview

1. create new virtual environment.
2. install necessary dependencies from requirements.txt file and activate the environment.
3. open the **sql_agent_for_pl_sql.ipynb** file.
4. run all the cells, Last cell will give you a user interface.

# Database connection

In [1]:
import os
from dotenv import load_dotenv

# load keys
load_dotenv()



True

In [2]:
# set your Database credentials 
db_user = os.getenv('db_user')
db_password = os.getenv('db_password')
db_host = os.getenv('db_host')
db_name = os.getenv('db_name')
db_port = os.getenv('db_port')


In [3]:
from langchain_community.utilities.sql_database import SQLDatabase

# Database connection
SQLALCHEMY_DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

db = SQLDatabase.from_uri(SQLALCHEMY_DATABASE_URL)

In [5]:
db.table_info

e:\sql_agents_learning\sql_env\lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "account_account, account_bank_statement, account_bank_statement_line, account_cash_rounding, account_fiscal_position, account_full_reconcile, account_group, account_incoterms, account_journal, account_move, account_move_line, account_partial_reconcile, account_payment, account_payment_method, account_payment_method_line, account_payment_term, account_reconcile_model, account_tax, account_tax_group, account_tax_repartition_line, base_partner_merge_automatic_wizard, base_partner_merge_line, calendar_event, calendar_recurrence, crm_team, discuss_channel_member, discuss_channel_rtc_session, ir_act_window, ir_attachment, ir_mail_server, ir_model, ir_module_category, ir_module_module, ir_sequence, ir_ui_view, l10n_in_port_code, mail_activity_type, mail_alias, mail_alias_domain, mail_template, payment_method,

'\nCREATE TABLE account_account (\n\tid SERIAL NOT NULL, \n\tcurrency_id INTEGER, \n\tcompany_id INTEGER NOT NULL, \n\tgroup_id INTEGER, \n\troot_id INTEGER, \n\tcreate_uid INTEGER, \n\twrite_uid INTEGER, \n\tcode VARCHAR(64) NOT NULL, \n\taccount_type VARCHAR NOT NULL, \n\tinternal_group VARCHAR, \n\tname JSONB NOT NULL, \n\tnote TEXT, \n\tdeprecated BOOLEAN, \n\tinclude_initial_balance BOOLEAN, \n\treconcile BOOLEAN, \n\tnon_trade BOOLEAN, \n\tcreate_date TIMESTAMP WITHOUT TIME ZONE, \n\twrite_date TIMESTAMP WITHOUT TIME ZONE, \n\tCONSTRAINT account_account_pkey PRIMARY KEY (id), \n\tCONSTRAINT account_account_company_id_fkey FOREIGN KEY(company_id) REFERENCES res_company (id) ON DELETE RESTRICT, \n\tCONSTRAINT account_account_create_uid_fkey FOREIGN KEY(create_uid) REFERENCES res_users (id) ON DELETE SET NULL, \n\tCONSTRAINT account_account_currency_id_fkey FOREIGN KEY(currency_id) REFERENCES res_currency (id) ON DELETE SET NULL, \n\tCONSTRAINT account_account_group_id_fkey FOREIGN 

# If you are getting a error here, Ask Server admin to grant access to your ip address.

In [9]:
db.dialect

'postgresql'

## llm and agent initialization and execution

In [20]:
import os
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType
from langchain_groq import ChatGroq
from dotenv import load_dotenv

# load keys
load_dotenv()

# set api for groq
os.environ["GROQ_API_KEY"] = os.getenv("groq_api_key")

# LLM
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=600,
    timeout=None,
    max_retries=2,
)

# agent creation
agent_executor = create_sql_agent(llm, db=db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [21]:
agent_executor.invoke('what is the total count of the tables in this database?')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: account_account, account_account_account_journal_rel, account_account_account_tag, account_account_tag, account_account_tag_account_move_line_rel, account_account_tag_account_tax_repartition_line_rel, account_account_tag_product_template_rel, account_account_tax_default_rel, account_accrued_orders_wizard, account_analytic_account, account_analytic_applicability, account_analytic_distribution_model, account_analytic_line, account_analytic_plan, account_automatic_entry_wizard, account_automatic_entry_wizard_account_move_line_rel, account_bank_statement, account_bank_statement_ir_attachment_rel, account_bank_statement_line, account_cash_rounding, account_financial_year_op, account_fiscal_position, account_fiscal_position_account, account_fiscal_position_res_country_state_rel, account_fiscal_position_tax, account

{'input': 'what is the total count of the tables in this database?',
 'output': 'There are 408 tables in the database.'}

# Add memory to the agent

In [22]:
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# To store previous chat conversation
memory = ChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)



In [23]:
agent_with_chat_history.invoke(
    {"input": "can you list some of the table name?"},
    config={"configurable": {"session_id": "<foo>"}},
)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: account_account, account_account_account_journal_rel, account_account_account_tag, account_account_tag, account_account_tag_account_move_line_rel, account_account_tag_account_tax_repartition_line_rel, account_account_tag_product_template_rel, account_account_tax_default_rel, account_accrued_orders_wizard, account_analytic_account, account_analytic_applicability, account_analytic_distribution_model, account_analytic_line, account_analytic_plan, account_automatic_entry_wizard, account_automatic_entry_wizard_account_move_line_rel, account_bank_statement, account_bank_statement_ir_attachment_rel, account_bank_statement_line, account_cash_rounding, account_financial_year_op, account_fiscal_position, account_fiscal_position_account, account_fiscal_position_res_country_state_rel, account_fiscal_position_tax, account

{'input': 'can you list some of the table name?',
 'chat_history': [],
 'output': 'The first 10 table names in the database are: base_import_mapping, base_import_import, pg_statistic, pg_type, base_import_module, ir_module_module_dependency, pg_foreign_table, pg_authid, pg_shadow, pg_statistic_ext_data'}

In [24]:
agent_with_chat_history.invoke(
    {"input": "what web_tour_tour table contains?"},
    config={"configurable": {"session_id": "<foo>"}},
)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: account_account, account_account_account_journal_rel, account_account_account_tag, account_account_tag, account_account_tag_account_move_line_rel, account_account_tag_account_tax_repartition_line_rel, account_account_tag_product_template_rel, account_account_tax_default_rel, account_accrued_orders_wizard, account_analytic_account, account_analytic_applicability, account_analytic_distribution_model, account_analytic_line, account_analytic_plan, account_automatic_entry_wizard, account_automatic_entry_wizard_account_move_line_rel, account_bank_statement, account_bank_statement_ir_attachment_rel, account_bank_statement_line, account_cash_rounding, account_financial_year_op, account_fiscal_position, account_fiscal_position_account, account_fiscal_position_res_country_state_rel, account_fiscal_position_tax, account

e:\sql_agents_learning\sql_env\lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "account_account, account_bank_statement, account_bank_statement_line, account_cash_rounding, account_fiscal_position, account_full_reconcile, account_group, account_incoterms, account_journal, account_move, account_move_line, account_partial_reconcile, account_payment, account_payment_method, account_payment_method_line, account_payment_term, account_reconcile_model, account_tax, account_tax_group, account_tax_repartition_line, base_partner_merge_automatic_wizard, base_partner_merge_line, calendar_event, calendar_recurrence, crm_team, discuss_channel_member, discuss_channel_rtc_session, ir_act_window, ir_attachment, ir_mail_server, ir_model, ir_module_category, ir_module_module, ir_sequence, ir_ui_view, l10n_in_port_code, mail_activity_type, mail_alias, mail_alias_domain, mail_template, payment_method,


CREATE TABLE web_tour_tour (
	id SERIAL NOT NULL, 
	user_id INTEGER, 
	name VARCHAR NOT NULL, 
	CONSTRAINT web_tour_tour_pkey PRIMARY KEY (id), 
	CONSTRAINT web_tour_tour_user_id_fkey FOREIGN KEY(user_id) REFERENCES res_users (id) ON DELETE SET NULL
)

/*
3 rows from web_tour_tour table:
id	user_id	name

*/Thought: I now know the columns in the web_tour_tour table.  I should query the table to see what data it contains.

Action: sql_db_query_checker
Action Input: SELECT name FROM web_tour_tour LIMIT 10;SELECT name FROM web_tour_tour LIMIT 10;Thought: The query is correct, I should execute it.

Action: sql_db_query
Action Input: SELECT name FROM web_tour_tour LIMIT 10;Question: what web_tour_tour table contains?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: account_account, account_account_account_journal_rel, account_account_account_tag, account_account

{'input': 'what web_tour_tour table contains?',
 'chat_history': [HumanMessage(content='can you list some of the table name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first 10 table names in the database are: base_import_mapping, base_import_import, pg_statistic, pg_type, base_import_module, ir_module_module_dependency, pg_foreign_table, pg_authid, pg_shadow, pg_statistic_ext_data', additional_kwargs={}, response_metadata={})],
 'output': 'The web_tour_tour table contains the columns id, user_id, and name.'}

# Fresh memory for gradio chatbot

In [26]:
# To store previous chat conversation
memory = ChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Gradio User Interface

In [27]:
from sqlalchemy.exc import ProgrammingError
import gradio as gr

def execute_query(question,  history: list[dict]):
    try:
        # Focus on the latest user message only
        query = str(question)

        # Generate SQL query from question
        response = agent_with_chat_history.invoke(
            {"input":{query}},
            config={"configurable": {"session_id": "<foo>"}},
            )
        response_text = response['output']

        # Append the assistant's response to the history
        response_final = {"role": "assistant", "content": response_text}

        # Return the query and the result
        yield response_final 
    except ProgrammingError as e:
        gr.Error(f"An error occurred: {e}")
        return None
    
# Create a Gradio ChatInterface using the respond function
demo = gr.ChatInterface(execute_query, type="messages")
# Launch the Gradio interface with debugging enabled
if __name__ == "__main__":
    demo.launch(debug=True)

e:\sql_agents_learning\sql_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: account_account, account_account_account_journal_rel, account_account_account_tag, account_account_tag, account_account_tag_account_move_line_rel, account_account_tag_account_tax_repartition_line_rel, account_account_tag_product_template_rel, account_account_tax_default_rel, account_accrued_orders_wizard, account_analytic_account, account_analytic_applicability, account_analytic_distribution_model, account_analytic_line, account_analytic_plan, account_automatic_entry_wizard, account_automatic_entry_wizard_account_move_line_rel, account_bank_statement, account_bank_statement_ir_attachment_rel, account_bank_statement_line, account_cash_rounding, account_financial_year_op, account_fiscal_position, account_fiscal_position_account, account_fiscal_position_res_country_state_rel, account_fiscal_position_tax, account

Error in RootListenersTracer.on_chain_end callback: ValueError("Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got {'what is the total count of tables in this database?'}.")


...

Thought: I have the list of table names in the database. I can now return the answer.

Final Answer: The first ten table names in the database are: base_import_mapping, base_import_import, pg_statistic, pg_type, base_import_module, ir_module_module_dependency, pg_foreign_table, pg_authid, pg_shadow, pg_statistic_ext_data.

> Finished chain.


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: account_account, account_account_account_journal_rel, account_account_account_tag, account_account_tag, account_account_tag_account_move_line_rel, account_account_tag_account_tax_repartition_line_rel, account_account_tag_product_template_rel, account_account_tax_default_rel, account_accrued_orders_wizard, account_analytic_account, account_analytic_applicability, account_analytic_distribution_model, account_analytic_line, account_analytic_plan, account_automatic_entry_wizard, account_automatic_entry_wizard_account_move_line_rel, account_bank_statement, account_bank_statement_ir_attachment_rel, account_bank_statement_line, account_cash_rounding, account_financial_year_op, account_fiscal_position, account_fiscal_position_account, account_fiscal_position_res_country_state_rel, account_fiscal_position_tax, account

Error in RootListenersTracer.on_chain_end callback: ValueError("Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got {'what are the columns contained in pg_statistic?'}.")


Thought: I now know the final answer
Final Answer: The columns contained in pg_statistic are: starelid, staattnum, stainherit, stanullfrac, stawidth, stadistinct, stakind1, stakind2, stakind3, stakind4, stakind5, staop1, staop2, staop3, staop4, staop5, stacoll1, stacoll2, stacoll3, stacoll4, stacoll5, stanumbers1, stanumbers2, stanumbers3, stanumbers4, stanumbers5, stavalues1, stavalues2, stavalues3, stavalues4, stavalues5.

> Finished chain.
Keyboard interruption in main thread... closing server.
